In [9]:
from experimental.enc_dec_dataset import EncDecDataset, CustomEncoder
from experimental.enc_dec_training import EncDecTrainer
from transformers import EncoderDecoderModel, AutoTokenizer

import torch
import json
import os
import os.path as op

model = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased", "bert-base-uncased",pad_token_id=0)
#orig_model = EncoderDecoderModel.from_pretrained("patrickvonplaten/bert2bert-cnn_dailymail-fp16", pad_token_id=0)
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", pad_token_id=0)
cust_enc = CustomEncoder(model.encoder)
model.encoder = cust_enc
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size
#cust_dec = CustomDecoder(model.decoder)

model.encoder = cust_enc


DATA_DIR = "DialogSum_Data"

SAMPLE_DATA = op.join(DATA_DIR, 'dialogsum.sample.jsonl')
TEST_DATA = op.join(DATA_DIR, 'dialogsum.test.jsonl')
TRAIN_DATA = op.join(DATA_DIR, 'dialogsum.train.jsonl')
DEV_DATA = op.join(DATA_DIR, 'dialogsum.dev.jsonl')

def load_jsonl(filepath):
    
    output_list = []
    
    with open(filepath) as sd_file:
        lines = sd_file.readlines()
    
    for line in lines:
        output_list.append(json.loads(line))

    return output_list


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


TypeError: __init__() got an unexpected keyword argument 'start_token_id'

In [ ]:
dialogue = ["this is a sentence", "what is the sentence about", "yes it is"]
tokens = tokenizer.batch_encode_plus(dialogue, padding=True, return_tensors="pt")



tokens.pop("attention_mask")
tokens["attention_mask"] = torch.Tensor([[1,1,1]])
tokens.pop("token_type_ids")
output = model.generate(**tokens)
print(tokenizer.batch_decode(output))



ValueError: `decoder_start_token_id` or `bos_token_id` has to be defined for encoder-decoder generation.

In [ ]:
tokenizer.decode(101)

'[CLS]'

In [ ]:


if __name__ == "__main__":




    model_params = {
        "TRAIN_BATCH_SIZE": 1,  # training batch size
        "VALID_BATCH_SIZE": 1,  # validation batch size
        "TRAIN_EPOCHS":  1,  # number of training epochs
        "VAL_EPOCHS": 1,  # number of validation epochs
        "LEARNING_RATE": 1e-3,  # learning rate
        "EXPERIMENT_NAME": "preliminary",
        #"MAX_SOURCE_TEXT_LENGTH": 512,  # max length of source text
        #"MAX_TARGET_TEXT_LENGTH": 50,  # max length of target text
        "SEED": 42,  # set seed for reproducibility
        }


    dev_data_list = load_jsonl(DEV_DATA)
    dev_dataset = EncDecDataset(dev_data_list, 
                  tokenizer=tokenizer)

    train_data_list = load_jsonl(TRAIN_DATA)
    train_dataset = EncDecDataset(train_data_list,
                        tokenizer=tokenizer)

    model = EncDecTrainer(train_dataset=train_dataset, val_dataset=dev_dataset,
                    model=model, tokenizer=tokenizer, model_params=model_params)
                  

NameError: name 'load_jsonl' is not defined